<a href="https://colab.research.google.com/github/prasann25/colab/blob/main/04_transfer_learning_in_tensorflow-part1_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning With TensorFlow Part 1: Feature Extraction

Transfer learning is leveraging a working model's existing architecture and learned patterns for our own problem.

There are 2 main benefits :
1. Can leverage an existing neural network architecture proven to work on problems similar to our own.
2. Can leverage a working neural network architecture which has already learned patterns on similar data to our own, then we can adapt those patterns to our own data.

In [1]:
# Are we using a GPU?
!nvidia-smi

Mon Jul 12 00:08:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading and becoming one with the data

In [2]:
# Get data(10% of food classes from Food101) https://www.kaggle.com/dansbecker/food-101
import zipfile

#Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()


--2021-07-12 00:08:22--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.203.128, 74.125.199.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   185MB/s    in 0.9s    

2021-07-12 00:08:23 (185 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [3]:
# How many images in each folder
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent") :
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.'")

There are 2 directories and 0 images in '10_food_classes_10_percent'.'
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.'
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.'
There are 0 directories and 250 images in '10_food_classes_10_p

## Creating data loaders(preparing the data)
We'll use the `ImageDataGenerator` class to load in out images in batches.

In [4]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SHAPE=(224, 224)
BATCH_SIZE=32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale = 1/255.)
test_datagen  = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir, 
                                                          target_size=IMG_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode="categorical")
print("Testing images:")
test_data = test_datagen.flow_from_directory(test_dir,
                                                        target_size=IMG_SHAPE,
                                                        batch_size=BATCH_SIZE,
                                                        class_mode="categorical")





Training images:
Found 750 images belonging to 10 classes.
Testing images:
Found 2500 images belonging to 10 classes.


In [5]:
len(train_data_10_percent), len(test_data)

(24, 79)

## Setting up callbacks (things to run whilst our model trains)

Callbacks are extra functionality you can add to your models to be performed during or after training. Some of the most popular callbacks :
* Tracking experiments with the TensorBoard callback
* Model checkpoint with the ModelCheckpoint callback
* Stopping a model from training( before it trains too long and overfits) with the EarlyStopping callback

In [6]:
# Create TensorBoard callback (functionized becaused we need to create a new one for each model)

import datetime

def create_tensorboard_callback(dir_name, experiment_name) :
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y&m&d-%H%M%S")
  tensorboard_callback = tf.keras.callback.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log file to: {log_dir}")
  return tensorboard_callback

## Creating Models using TensorFlow Hub

In the past we'v e used TensorFlow to create our own models layer by layer from scratch.

Now we're going to do a similar process, except the majority of our model's layers are going to come from TensorFlow Hub.
We can access pre-trained models on [https://www.tensorflow.org/hub](https://www.tensorflow.org/hub)


Research paper on which architecture is performing well on different classfication models
[https://paperswithcode.com](https://paperswithcode.com/)

https://paperswithcode.com/task/image-classification

Browsing the TensorFlow Hub page and sorting for image classification, we found the following vector model link : https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1



In [7]:
import tensorflow_hub as hub